# ME314 Homework 4

###Submission instructions

Deliverables that should be included with your submission are shown in **bold** at the end of each problem statement and the corresponding supplemental material. **Your homework will be graded IFF you submit a single PDF, .mp4 videos of animations when requested and a link to a Google colab file that meet all the requirements outlined below.**

- List the names of students you've collaborated with on this homework assignment.
- Include all of your code (and handwritten solutions when applicable) used to complete the problems.
- Highlight your answers (i.e. **bold** and outline the answers) for handwritten or markdown questions and include simplified code outputs (e.g. .simplify()) for python questions.

- Enable Google Colab permission for viewing 
 * Click Share in the upper right corner
 * Under "Get Link" click "Share with..." or "Change" 
 * Then make sure it says "Anyone with Link" and "Editor" under the dropdown menu
- Make sure all cells are run before submitting (i.e. check the permission by running your code in a private mode)
 * Please don't make changes to your file after submitting, so we can grade it!
- Submit a link to your Google Colab file that has been run (before the submission deadline) and don't edit it afterwards!

**NOTE:** This Juputer Notebook file serves as a template for you to start homework. Make sure you first copy this template to your own Google driver (click "File" -> "Save a copy in Drive"), and then start to edit it.

In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

In [2]:
##############################################################################################
# If you're using Google Colab, uncomment this section by selecting the whole section and press
# ctrl+'/' on your and keyboard. Run it before you start programming, this will enable the nice 
# LaTeX "display()" function for you. If you're using the local Jupyter environment, leave it alone
##############################################################################################
# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.1.1/latest.js?config=TeX-AMS_HTML"
#     javascript(url=url)
#     return sym.printing.latex(exp,**options)
# sym.init_printing(use_latex="mathjax",latex_printer=custom_latex_printer)

# Helper Functions

In [25]:
def solve_EL(eqn, var):
    '''
    Helper function to solve and display the solution for the Euler-Lagrange
    equations.
    
    Inputs:
    - eqn: Euler-Lagrange equation (type: Sympy Equation())
    - var: state vector (type: Sympy Matrix). typically a form of q-doubledot
        but may have different terms
    
    Outputs:
    - Prints symbolic solutions
    - Returns symbolic solutions in a dictionary
    '''
    
    soln = sym.solve(eqn, var, dict = True)
    eqns_solved = []
    
    
    for i, sol in enumerate(soln):
        for x in list(sol.keys()):
            eqn_solved = sym.Eq(x, sol[x])
            eqns_solved.append(eqn_solved)
            
    return eqns_solved

def solve_constrained_EL(lamb, phi, q, lhs):
    """Now uses just the LHS of the constrained E-L equations,
    rather than the full equation form"""
    
    qd = q.diff(t)
    qdd = qd.diff(t)
    
    phidd = phi.diff(t).diff(t)
    lamb_grad = sym.Matrix([lamb * phi.diff(a) for a in q])
    q_mod = qdd.row_insert(2, sym.Matrix([lamb]))
    
    #format equations so they're all in one matrix
    expr_matrix = lhs - lamb_grad
    phidd_matrix = sym.Matrix([phidd])
    expr_matrix = expr_matrix.row_insert(2,phidd_matrix)
    
    print("Expressions to be solved:")
    display(expr_matrix)
    print("Variables to solve for:")
    display(q_mod)

    #solve E-L equations
    eqns_solved = solve_EL(expr_matrix, q_mod)
    return eqns_solved


In [39]:
def rk4(dxdt, x, t, dt):
    '''
    Applies the Runge-Kutta method, 4th order, to a sample function,
    for a given state q0, for a given step size. Currently only
    configured for a 2-variable dependent system (x,y).
    ==========
    dxdt: a Sympy function that specifies the derivative of the system of interest
    t: the current timestep of the simulation
    x: current value of the state vector
    dt: the amount to increment by for Runge-Kutta
    ======
    returns:
    x_new: value of the state vector at the next timestep
    '''  
    k1 = dt * dxdt(t, x)
    k2 = dt * dxdt(t + dt/2.0, x + k1/2.0)
    k3 = dt * dxdt(t + dt/2.0, x + k2/2.0)
    k4 = dt * dxdt(t + dt, x + k3)
    x_new = x + (k1 + 2.0*k2 + 2.0*k3 + k4)/6.0
    
    return x_new
    
rk = rk4(lambda t, x: x**2, 0.5, 2, 0.1)
assert np.isclose(rk, 0.526315781526278075), f"RK4 value: {rk}" #from an online RK4 solver
print("assertion passed")


assertion passed


In [41]:
def simulate(f, x0, tspan, dt, integrate):
    """
    This function takes in an initial condition x0, a timestep dt,
    a time span tspan consisting of a list [min_time, max_time],
    as well as a dynamical system f(x) that outputs a vector of the
    same dimension as x0. It outputs a full trajectory simulated
    over the time span of dimensions (xvec_size, time_vec_size).
    
    Parameters
    ============
    f: Python function
        derivate of the system at a given step x(t), 
        it can considered as \dot{x}(t) = func(x(t))
    x0: NumPy array
        initial conditions
    tspan: Python list
        tspan = [min_time, max_time], it defines the start and end
        time of simulation
    dt:
        time step for numerical integration
    integrate: Python function
        numerical integration method used in this simulation

    Return
    ============
    x_traj:
        simulated trajectory of x(t) from t=0 to tf
    """
    N = int((max(tspan)-min(tspan))/dt)
    x = np.copy(x0)
    tvec = np.linspace(min(tspan),max(tspan),N)
    xtraj = np.zeros((len(x0),N))
    
    for i in range(N):
        t = tvec[i]
        xtraj[:,i]=integrate(f,x,t,dt)
        x = np.copy(xtraj[:,i])
    return xtraj 
      

In [3]:
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='https://github.com/MuchenSun/ME314pngs/raw/master/dynhoop2.png' width=500' height='350'></table>"))

## Problem 1 (20pts)

Take the bead on a hoop example shown in the image above, model it using a torque input $\tau$ (about the vertical $z$ axis) instead of a velocity input $\omega$. You will need to add a configuration variable $\psi$ that is the rotation about the $z$ axis, so that the system configuration vector is $q=[\theta,\psi]$. Use Python's SymPy package to compute the equations of motion for this system in terms of $\theta, \psi$.

*Hint 1: Note that this should be a Lagrangian system with an external force.*

**Turn in: A copy of code used to symbolically solve for the equations of motion, also include the code outputs, which should be the equations of motion.**

## Problem 2 (30pts)

Consider a point mass in 3D space under the forces of gravity and a radial spring from the origin. The system's Lagrangian is:
$$
L=\frac{1}{2} m (\dot{x}^2+\dot{y}^2+\dot{z}^2)- \frac{1}{2} k (x^2+y^2+z^2)-mgz
$$
Consider the following rotation matrices, defining rotations about the $z$, $y$, and $x$ axes respectively:
$$
R_\theta =\begin{bmatrix}\cos \theta & -\sin \theta & 0 \\ \sin \theta &
    \cos \theta& 0 \\ 0 & 0 & 1\end{bmatrix}, 
    \ \
R_\psi = \begin{bmatrix}\cos \psi & 0 & \sin \psi \\ 0 & 1 & 0 \\ 
    -\sin \psi & 0 & \cos \psi \end{bmatrix}, \ \
R_\phi = \begin{bmatrix} 1 & 0 & 0 \\ 0 & \cos \phi & -\sin \phi \\ 
    0 & \sin \phi & \cos \phi \end{bmatrix} 
$$ 

and answer the following three questions:

1. Which, if any, of the transformations $q_{\theta}=R_{\theta}q$, $q_{\psi}=R_{\psi}q$, or $q_{\phi}=R_{\phi}q$ keeps the Lagrangian fixed (invariant)? Is this invariance global or local?

2. Use small angle approximations to linearize your transformation(s) from the first question. The resulting new transformation(s) should have the form $q_{\epsilon}=q+\epsilon G(q)$. Compute the difference in the Lagrangian $L(q_\epsilon, \dot{q}_\epsilon) - L(q,\dot{q})$ through this/these transformation(s).

3. Apply Noether's theorem to determine a conserved quantity. What does this quantity represent physically?  Is there any rationale behind its conservation?

You can solve this problem by hand or use Python's SymPy to do the symbolic computation for you.

*Hint 1: For question (1), try to imagine how this system looks. Even though the $x$, $y$, and $z$ axes seem to have the same influence on the system, rotation around some axes will influence the Lagrangian more than others will.*

*Hint 2: Global invariance here means for any magnitude of rotation the Lagrangian will remain fixed.*

**Turn in: A scanned (or photograph from your phone or webcam) copy of your hand written solution. You can also use $\LaTeX$. If you use SymPy, then you just need to include a copy of code and the code outputs, with notes that explain why the code outputs can answer the questions.**

In [ ]:
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='https://github.com/atulletaylor/ME314Figures/raw/main/hw4problem3.png' width='600' height='350'></td></tr></table>"))

""


## Problem 3 (20pts)

For the inverted cart-pendulum system in Homework 1 (feel free to make use of the provided solutions), compute the conserved momentum using N&ouml;ther's theorem.  Plot the momentum for the same simulation parameters and initial conditions.  Taking into account the conserved quantities, what is the minimal number of *states* in the cart/pendulum system that can vary?  (Hint: In some coordinate systems it may *look* like all the states are varying, but if you choose your coordinates cleverly fewer states will vary.)

**Turn in: A copy of code used to calculate the conserved quantity and your answer to the question. You don't need to turn in equations of motion, but you need to include the plot of the conserved quantity evaluated along the system trajectory.**

In [ ]:
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='https://github.com/atulletaylor/ME314Figures/raw/main/hw4p4.png' width='600' height='350'></td></tr></table>"))

""


In [16]:
#equations drawn from hw1

#1. grab variables
t = sym.symbols(r't') #ind. var.
R, m, M, g = sym.symbols(r'R, m, M, g') #constants

const_dict = {
    M: 2,
    m: 1,
    g: 9.8,
    R: 1
}

#2. define position, velocity, accel for masses 1 and 2 as functions
xm = sym.Function(r'x_m')(t)
theta = sym.Function(r'\theta')(t)

xmd = xm.diff(t)
xmdd = xmd.diff(t)

thetad = theta.diff(t)
thetadd = thetad.diff(t)

#intermediate variables: x and y of mass on pendulum
xp = xm + R * sym.sin(theta)
yp = R * sym.cos(theta)

xpd = xp.diff(t)
ypd = yp.diff(t)

#kinetic and potential energy terms

KE_M = 0.5 * M * xmd**2
KE_m = 0.5 * m * (xpd**2 + ypd**2)
KE_sys = KE_M + KE_m
U_sys = m * g * yp

print("Lagrangian function:")
lagrangian = KE_sys - U_sys
lagrangian = lagrangian.simplify()
display(lagrangian)

Lagrangian function:


0.5*M*Derivative(x_m(t), t)**2 - R*g*m*cos(\theta(t)) + 0.5*m*(R**2*Derivative(\theta(t), t)**2 + 2*R*cos(\theta(t))*Derivative(\theta(t), t)*Derivative(x_m(t), t) + Derivative(x_m(t), t)**2)

In [18]:
#set up conserved quantities calculations for each element in state vector

#define state of system
q = sym.Matrix([xm, theta])
qd = q.diff(t)
qdd = qd.diff(t)

n = len(q)
pdot_matrix = sym.zeros(n,1)
dLdQ_matrix = sym.zeros(n,1)

for i, x in enumerate(q):
    xd = qd[i]
    pdot_matrix[i] = lagrangian.diff(xd).diff(t)
    dLdQ_matrix[i] = lagrangian.diff(x)
    
print("\nd/dt(dL/dqd) terms for qd = (x, theta):")
display(sym.simplify(pdot_matrix))
print("\ndL/dq terms for qd = (x, theta):")
display(sym.simplify(dLdQ_matrix))


d/dt(dL/dqd) terms for qd = (x, theta):


Matrix([
[1.0*M*Derivative(x_m(t), (t, 2)) + m*(-R*sin(\theta(t))*Derivative(\theta(t), t)**2 + R*cos(\theta(t))*Derivative(\theta(t), (t, 2)) + Derivative(x_m(t), (t, 2)))],
[             1.0*R*m*(R*Derivative(\theta(t), (t, 2)) - sin(\theta(t))*Derivative(\theta(t), t)*Derivative(x_m(t), t) + cos(\theta(t))*Derivative(x_m(t), (t, 2)))]])


dL/dq terms for qd = (x, theta):


Matrix([
[                                                                          0],
[R*m*(g - 1.0*Derivative(\theta(t), t)*Derivative(x_m(t), t))*sin(\theta(t))]])

Conserved quantity, therefore, is momentum of the cart in the x direction. The expression for the conserved quantity is given by:

In [20]:
conserved = pdot_matrix[0]
eq = sym.Eq(conserved, 0)
display(eq)

Eq(1.0*M*Derivative(x_m(t), (t, 2)) + 0.5*m*(-2*R*sin(\theta(t))*Derivative(\theta(t), t)**2 + 2*R*cos(\theta(t))*Derivative(\theta(t), (t, 2)) + 2*Derivative(x_m(t), (t, 2))), 0)

The minimal number of states in the system that can vary are $\theta_m$ and $x_M$. Any Cartesian coordinate system for the mass would give an inaccurate number of states, as x and y of the smaller mass can be determined from x of the smaller mass and theta.

## Problem 4 (30 pts)
Using the same inverted cart pendulum system, add a constraint such that the pendulum follows the path of a parabola with a vertex of $(1,0)$. 

Then, simulate the system using $x$ and $\theta$ as the configuration variables for $t\in[0,15]$ with $dt=0.01$. The constants are $M=2, m=1, R=1, g=9.8$. Use the initial conditions $x=0, \theta=0, \dot{x}=0, \dot{\theta}=0.01$ for your simulation.

You should use the Runge–Kutta integration function provided in previous homework for simulation. Plot the simulated trajectory for $x, \theta$ versus time. We have a provided an animation function for testing. 

*Hint 1: You will need the time derivatives of $\phi$ to solve the system of equations.*

*Hint 2: Make sure to be solving for $\lambda$ at the same time as your equations of motion.*

*Hint 3: Note that if you make your initial condition velocities faster or dt lower resolution, you may not be able to simulate the system because this is challenging constraint.*

**Turn in: A copy of code used to simulate the system, you don't need to turn in equations of motion, but you need to include the plot of the simulated trajectories.**

In [28]:
#set up left hand side of constrained Euler-Lagrange equations
EL_lhs = pdot_matrix - dLdQ_matrix
lamb = sym.symbols(r'\lambda')

#let constraint be approximately (x-1) =-y^2
phi = xp - 1 + yp**2 #equals 0

#need gradient of constraint for RHS of equation and for additional constraint EQ
eqns_solved = solve_constrained_EL(lamb, phi, q, EL_lhs)

Expressions to be solved:


Matrix([
[                                                                                                                                               1.0*M*Derivative(x_m(t), (t, 2)) - \lambda + 0.5*m*(-2*R*sin(\theta(t))*Derivative(\theta(t), t)**2 + 2*R*cos(\theta(t))*Derivative(\theta(t), (t, 2)) + 2*Derivative(x_m(t), (t, 2)))],
[-R*g*m*sin(\theta(t)) + 1.0*R*m*sin(\theta(t))*Derivative(\theta(t), t)*Derivative(x_m(t), t) - \lambda*(-2*R**2*sin(\theta(t))*cos(\theta(t)) + R*cos(\theta(t))) + 0.5*m*(2*R**2*Derivative(\theta(t), (t, 2)) - 2*R*sin(\theta(t))*Derivative(\theta(t), t)*Derivative(x_m(t), t) + 2*R*cos(\theta(t))*Derivative(x_m(t), (t, 2)))],
[                        2*R**2*sin(\theta(t))**2*Derivative(\theta(t), t)**2 - 2*R**2*sin(\theta(t))*cos(\theta(t))*Derivative(\theta(t), (t, 2)) - 2*R**2*cos(\theta(t))**2*Derivative(\theta(t), t)**2 - R*sin(\theta(t))*Derivative(\theta(t), t)**2 + R*cos(\theta(t))*Derivative(\theta(t), (t, 2)) + Derivative(x_m(t), (t, 2))]])

Variables to solve for:


Matrix([
[   Derivative(x_m(t), (t, 2))],
[Derivative(\theta(t), (t, 2))],
[                      \lambda]])

In [31]:
new_eqs = []
for i, eq in enumerate(eqns_solved):
    
    if i == 2:
    new_eq = eq.simplify()
    new_eqs.append(new_eq)
    display(new_eq)

Eq(Derivative(x_m(t), (t, 2)), m*(4.0*R**3*sin(\theta(t))**4*Derivative(\theta(t), t)**2 - 8.0*R**3*sin(\theta(t))**2*Derivative(\theta(t), t)**2 + 4.0*R**3*Derivative(\theta(t), t)**2 - 2.0*R**2*sin(\theta(t))**3*Derivative(\theta(t), t)**2 + 2.0*R*g*sin(\theta(t))*cos(\theta(t)) + 1.0*R*Derivative(\theta(t), t)**2 - 1.0*g*cos(\theta(t)))*sin(\theta(t))/(0.5*M*R**2*(1 - cos(4*\theta(t))) - 4.0*M*R*sin(\theta(t))*cos(\theta(t))**2 + M*cos(\theta(t))**2 + 0.5*R**2*m*(1 - cos(4*\theta(t))) - m*cos(\theta(t))**2 + m))

Eq(Derivative(\theta(t), (t, 2)), (-1.0*M*R**3*sin(4*\theta(t))*Derivative(\theta(t), t)**2 + 0.5*M*R**2*cos(\theta(t))*Derivative(\theta(t), t)**2 + 1.5*M*R**2*cos(3*\theta(t))*Derivative(\theta(t), t)**2 + 0.5*M*R*sin(2*\theta(t))*Derivative(\theta(t), t)**2 - 1.0*R**3*m*sin(4*\theta(t))*Derivative(\theta(t), t)**2 - 0.5*R*m*sin(2*\theta(t))*Derivative(\theta(t), t)**2 + 1.0*g*m*sin(\theta(t)))/(R*(0.5*M*R**2*(1 - cos(4*\theta(t))) - 4.0*M*R*sin(\theta(t))*cos(\theta(t))**2 + M*cos(\theta(t))**2 + 0.5*R**2*m*(1 - cos(4*\theta(t))) - m*cos(\theta(t))**2 + m)))

Eq(\lambda, m*(2.0*M*R**2*cos(2*\theta(t))*Derivative(\theta(t), t)**2 + 0.5*M*R*g*cos(\theta(t)) - 0.5*M*R*g*cos(3*\theta(t)) + 1.0*M*R*sin(\theta(t))*Derivative(\theta(t), t)**2 - 0.5*M*g*sin(2*\theta(t)) - 0.5*R**2*m*(1 - cos(2*\theta(t)))**2*Derivative(\theta(t), t)**2 + 0.5*R*g*m*cos(\theta(t)) - 0.5*R*g*m*cos(3*\theta(t)))/(0.5*M*R**2*(1 - cos(4*\theta(t))) - 4.0*M*R*sin(\theta(t))*cos(\theta(t))**2 + M*cos(\theta(t))**2 + 0.5*R**2*m*(1 - cos(4*\theta(t))) - m*cos(\theta(t))**2 + m))

In [47]:
#substitute in variables and lambdify
xdd_sy = new_eqs[0].rhs.subs(const_dict)
thetadd_sy = new_eqs[1].rhs.subs(const_dict)

q_ext = sym.Matrix([xm, theta, xmd, thetad])

xdd_np = sym.lambdify(q_ext, xdd_sy)
thetadd_np = sym.lambdify(q_ext, thetadd_sy)


In [49]:
#simulate motion of system
def dxdt_p3(t, s):
    return np.array([s[2],s[3], xdd_np(*s), thetadd_np(*s)])

x0 = [0, 0, 0, 0.01]
tspan = [0,15]
dt = 0.01

q_array = simulate(dxdt_p3, x0, tspan, dt, rk4)

In [50]:
print(q_array)

[[-8.16747488e-09 -6.53556090e-08 -2.20636083e-07 ...  7.61583247e-01
   8.02052798e-01  8.42266266e-01]
 [ 1.00018171e-04  2.00105399e-04  3.00310827e-04 ...  3.58194666e+00
   3.60589272e+00  3.62943791e+00]
 [-2.45046508e-06 -9.80502298e-06 -2.20702267e-05 ...  4.05984387e+00
   4.03411115e+00  4.00861726e+00]
 [ 1.00044518e-02  1.00138129e-02  1.00280938e-02 ...  2.41623850e+00
   2.37378936e+00  2.33598433e+00]]


In [ ]:
x_array = q_array[0]
theta_array = q_array[1]
plt.plot(x_array)
plt.plot(theta_array)

plt.xlabel("Time (* 10^-2 seconds)")
plt.ylabel("Postion & Angle (m & radians)")
plt.title("Position of Double Pendulum over Time")

In [ ]:
def animate_cart_pend(traj_array,R=1,T=15):
    """
    Function to generate web-based animation of double-pendulum system

    Parameters:
    ================================================
    traj_array:
        trajectory of theta and x, should be a NumPy array with
        shape of (2,N)
    R:
        length of the pendulum
    T:
        length/seconds of animation duration

    Returns: None
    """

    ################################
    # Imports required for animation.
    from plotly.offline import init_notebook_mode, iplot
    from IPython.display import display, HTML
    import plotly.graph_objects as go

    #######################
    # Browser configuration.
    def configure_plotly_browser_state():
        import IPython
        display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
              requirejs.config({
                paths: {
                  base: '/static/base',
                  plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
                },
              });
            </script>
            '''))
    configure_plotly_browser_state()
    init_notebook_mode(connected=False)

    ###############################################
    # Getting data from pendulum angle trajectories.
    xcart=traj_array[0]
    ycart = 0.0*np.ones(traj_array[0].shape)
    N = len(traj_array[1])

    xx1=xcart+R*np.sin(traj_array[1])
    yy1=R*np.cos(traj_array[1])

     # Need this for specifying length of simulation

    ####################################
    # Using these to specify axis limits.
    xm=-4
    xM= 4
    ym=-4
    yM= 4

    ###########################
    # Defining data dictionary.
    # Trajectories are here.
    data=[
          dict(x=xcart, y=ycart, 
               mode='markers', name='Cart Traj', 
               marker=dict(color="green", size=2)
              ),
          dict(x=xx1, y=yy1, 
               mode='lines', name='Arm', 
               line=dict(width=2, color='blue')
              ),
           dict(x=xx1, y=yy1, 
               mode='lines', name='Pendulum',
               line=dict(width=2, color='purple')
              ),

           dict(x=xx1, y=yy1, 
               mode='markers', name='Pendulum Traj', 
               marker=dict(color="purple", size=2)
              ),
        ]

    ################################
    # Preparing simulation layout.
    # Title and axis ranges are here.
    layout=dict(xaxis=dict(range=[xm, xM], autorange=False, zeroline=False,dtick=1),
                yaxis=dict(range=[ym, yM], autorange=False, zeroline=False,scaleanchor = "x",dtick=1),
                title='Cart Pendulum Simulation', 
                hovermode='closest',
                updatemenus= [{'type': 'buttons',
                               'buttons': [{'label': 'Play','method': 'animate',
                                            'args': [None, {'frame': {'duration': T, 'redraw': False}}]},
                                           {'args': [[None], {'frame': {'duration': T, 'redraw': False}, 'mode': 'immediate',
                                            'transition': {'duration': 0}}],'label': 'Pause','method': 'animate'}
                                          ]
                              }]
               )

    ########################################
    # Defining the frames of the simulation.
    # This is what draws the lines from
    # joint to joint of the pendulum.
    frames=[dict(data=[go.Scatter(
                            x=[xcart[k]],
                            y=[ycart[k]],
                            mode="markers",
                            marker_symbol="square",
                            marker=dict(color="blue", size=30)),
                       
                       dict(x=[xx1[k],xcart[k]], 
                            y=[yy1[k],ycart[k]], 
                            mode='lines',
                            line=dict(color='red', width=3)
                            ),
                       go.Scatter(
                            x=[xx1[k]],
                            y=[yy1[k]],
                            mode="markers",
                            marker=dict(color="blue", size=12)),

                      ]) for k in range(N)]

    #######################################
    # Putting it all together and plotting.
    figure1=dict(data=data, layout=layout, frames=frames)           
    iplot(figure1)